## Unit 07. 異常檢測 (Anomaly Detection) 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P2_Unit07_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_2' / 'Unit07'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn套件
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定matplotlib中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 套件載入完成")

---
### **I. 練習題: 建立異常檢測模型**
 - 使用下面化工製程監控數據集。

In [ ]:
# 設定隨機種子
np.random.seed(42)

# 生成正常操作數據
n_normal = 1800
normal_data = {
    '反應器溫度': np.random.normal(180, 5, n_normal),
    '反應器壓力': np.random.normal(10, 1, n_normal),
    '進料流量': np.random.normal(30, 3, n_normal),
    '冷卻水流量': np.random.normal(50, 5, n_normal),
    '攪拌速度': np.random.normal(300, 20, n_normal),
    '產品濃度': np.random.normal(85, 3, n_normal),
    'pH值': np.random.normal(7, 0.3, n_normal),
    '催化劑濃度': np.random.normal(2.5, 0.2, n_normal)
}
df_normal = pd.DataFrame(normal_data)
df_normal['標籤'] = 0  # 0 表示正常

# 生成異常操作數據（約10%）
n_anomaly = 200

# 異常類型1: 溫度過高 (50筆)
anomaly_1 = {
    '反應器溫度': np.random.normal(220, 5, 50),
    '反應器壓力': np.random.normal(10, 1, 50),
    '進料流量': np.random.normal(30, 3, 50),
    '冷卻水流量': np.random.normal(50, 5, 50),
    '攪拌速度': np.random.normal(300, 20, 50),
    '產品濃度': np.random.normal(75, 5, 50),
    'pH值': np.random.normal(7, 0.3, 50),
    '催化劑濃度': np.random.normal(2.5, 0.2, 50)
}

# 異常類型2: 壓力異常 (50筆)
anomaly_2 = {
    '反應器溫度': np.random.normal(180, 5, 50),
    '反應器壓力': np.random.normal(15, 1, 50),
    '進料流量': np.random.normal(30, 3, 50),
    '冷卻水流量': np.random.normal(50, 5, 50),
    '攪拌速度': np.random.normal(300, 20, 50),
    '產品濃度': np.random.normal(80, 5, 50),
    'pH值': np.random.normal(7, 0.3, 50),
    '催化劑濃度': np.random.normal(2.5, 0.2, 50)
}

# 異常類型3: pH值異常 (50筆)
anomaly_3 = {
    '反應器溫度': np.random.normal(180, 5, 50),
    '反應器壓力': np.random.normal(10, 1, 50),
    '進料流量': np.random.normal(30, 3, 50),
    '冷卻水流量': np.random.normal(50, 5, 50),
    '攪拌速度': np.random.normal(300, 20, 50),
    '產品濃度': np.random.normal(70, 5, 50),
    'pH值': np.random.normal(5.5, 0.5, 50),
    '催化劑濃度': np.random.normal(2.5, 0.2, 50)
}

# 異常類型4: 多變數異常 (50筆)
anomaly_4 = {
    '反應器溫度': np.random.normal(195, 5, 50),
    '反應器壓力': np.random.normal(12, 1, 50),
    '進料流量': np.random.normal(40, 3, 50),
    '冷卻水流量': np.random.normal(35, 5, 50),
    '攪拌速度': np.random.normal(250, 20, 50),
    '產品濃度': np.random.normal(65, 5, 50),
    'pH值': np.random.normal(6.5, 0.3, 50),
    '催化劑濃度': np.random.normal(3.2, 0.2, 50)
}

# 合併異常數據
df_anomaly_1 = pd.DataFrame(anomaly_1)
df_anomaly_2 = pd.DataFrame(anomaly_2)
df_anomaly_3 = pd.DataFrame(anomaly_3)
df_anomaly_4 = pd.DataFrame(anomaly_4)

df_anomaly_1['標籤'] = 1  # 1 表示異常
df_anomaly_2['標籤'] = 1
df_anomaly_3['標籤'] = 1
df_anomaly_4['標籤'] = 1

# 合併所有數據
df = pd.concat([df_normal, df_anomaly_1, df_anomaly_2, df_anomaly_3, df_anomaly_4], ignore_index=True)

# 隨機打亂數據
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 保存數據
df.to_csv(DATA_DIR/'reactor_monitoring.csv', index=False, encoding='utf-8-sig')
print(f"數據集已生成: {DATA_DIR}/reactor_monitoring.csv")
print(f"數據形狀: {df.shape}")
print(f"正常樣本數: {(df['標籤']==0).sum()}")
print(f"異常樣本數: {(df['標籤']==1).sum()}")
print(f"異常比例: {(df['標籤']==1).sum()/len(df)*100:.2f}%")
print(f"\n前5筆數據:\n{df.head()}")

### **1. 數據準備與探索**
- 載入數據並檢查基本統計資訊
- 檢查異常樣本的分布
- 繪製特徵分布圖（至少3個特徵）
- 繪製相關性熱圖
- 數據分割: 訓練集80%、測試集20%
- **對特徵進行StandardScaler標準化（僅使用訓練集的參數）**

### **2. 建立 Isolation Forest 模型**
- 設定模型參數（contamination, n_estimators, random_state等）
- 在訓練集上進行訓練（僅使用正常數據或全部數據，依實際情況決定）
- 在測試集上進行預測
- 計算評估指標：Precision, Recall, F1-Score
- 繪製混淆矩陣
- 保存模型

### **3. 建立 One-Class SVM 模型**
- 設定模型參數（kernel, nu, gamma等）
- 在訓練集上進行訓練
- 在測試集上進行預測
- 計算評估指標：Precision, Recall, F1-Score
- 繪製混淆矩陣
- 保存模型

### **4. 建立 LOF (Local Outlier Factor) 模型**
- 設定模型參數（n_neighbors, contamination等）
- 在訓練集上進行訓練
- 在測試集上進行預測（注意：LOF需要使用fit_predict或重新fit）
- 計算評估指標：Precision, Recall, F1-Score
- 繪製混淆矩陣
- 保存模型

### **5. 建立 Elliptic Envelope 模型**
- 設定模型參數（contamination, support_fraction等）
- 在訓練集上進行訓練
- 在測試集上進行預測
- 計算評估指標：Precision, Recall, F1-Score
- 繪製混淆矩陣
- 保存模型

### **6. 模型綜合評估與比較**
- 製作評估指標比較表格（包含所有4個模型的Precision, Recall, F1-Score）
- 繪製ROC曲線比較圖（如果可以計算異常分數）
- 分析各模型的優缺點
- 選出最適合此數據集的模型並說明原因

---
## II. 額外加分作業 (自由繳交)
- 學生可自由選擇是否繳交加分作業. (下週上課前完成email電子檔)
- 分數會加在最後總成績上, 每份作業加0.1 ~ 1.0分.
- 分數加的不多, 真的不一定要交加分作業, 正常出席上課做好期末報告即可.
- 加分作業不一定要全部都做完, 想繳交至少完成其中一項實驗即可.
- 請務必自行完成! 你可以問AI, 問同學, 問學長姊, 甚至參考以前別人的作業, 但一定要自行[吸收][執行][整理]結果!
- 不要貼AI的答案寄給老師看, 你自己看就好.
- 如果系統自動比對發現作業內容(與他人雷同, 原創性低, 抄襲比例過高), 作業的分數會0分.
- 如果你直接100%複製別人的作業繳交, 你會直接被當掉(提供作業給別人複製的也一樣).
- 老師看你作業要花時間, 真的有心想寫加分作業, 請好好自己做.

---
### 使用課堂作業所提供的化工製程監控數據集
- 嘗試修改生成模擬數據的程式
- 改變參數, 改變異常類型, 添加特徵 ... 等等.
- 產生獨一無二屬於你的數據集
- 建立異常檢測模型來識別製程異常
- 然後開始進行下面實驗

### **實驗1: Contamination參數影響**

針對Isolation Forest和Elliptic Envelope，比較以下4種contamination值:
- 0.05 (5%異常)
- 0.10 (10%異常)
- 0.15 (15%異常)
- 0.20 (20%異常)

**要求**:
- 使用相同的訓練數據
- 記錄每個設定的Precision、Recall、F1-Score
- 製作比較表格和圖表
- **分析**: contamination參數如何影響模型性能? 如何選擇合適的contamination值?
- **思考**: 實際應用中，如果不知道真實異常比例該怎麼辦?

### **實驗2: LOF n_neighbors參數影響**

比較以下4種n_neighbors值:
- 5 (小鄰域)
- 10
- 20
- 50 (大鄰域)

**要求**:
- 固定contamination參數
- 記錄每個設定的性能指標
- 觀察模型對局部異常和全局異常的檢測能力
- 製作比較表格和圖表
- **分析**: n_neighbors參數如何影響模型的敏感度?
- **思考**: 小鄰域和大鄰域各適合什麼樣的異常類型?

### **實驗3: One-Class SVM Kernel影響**

比較以下4種kernel:
- 'linear'
- 'poly' (degree=3)
- 'rbf' (徑向基函數)
- 'sigmoid'

**要求**:
- 固定nu參數和其他設定
- 記錄每個kernel的性能指標和訓練時間
- 觀察不同kernel對非線性異常的檢測能力
- 製作比較表格和圖表
- **分析**: 哪種kernel最適合化工製程數據? 為什麼?
- **思考**: 如何根據數據特性選擇合適的kernel?

### **實驗4: 訓練策略比較**

比較以下3種訓練策略:
- **純正常數據訓練**: 僅使用標籤=0的數據訓練
- **全部數據訓練**: 使用全部數據（包含異常）訓練
- **95%正常數據訓練**: 移除5%極端正常值後訓練

**要求**:
- 選擇一個表現較好的模型進行實驗
- 記錄每種策略的性能指標
- 分析不同訓練策略對模型的影響
- 製作比較表格和圖表
- **分析**: 哪種訓練策略最適合實際應用?
- **思考**: 在實際應用中，如何獲得"純正常"的訓練數據?

### 💭 思考題

1. **異常檢測 vs 分類**: 異常檢測和二元分類（正常/異常）有什麼本質區別?
2. **評估指標選擇**: 在化工製程監控中，Precision和Recall哪個更重要? 為什麼?
3. **Contamination參數**: 如果實際異常比例是5%，但設定contamination=0.15會有什麼影響?
4. **模型選擇**: 在以下場景中，你會選擇哪個異常檢測模型?
   - 高維數據（特徵>20）
   - 小樣本數據（<500筆）
   - 實時監控（需要快速預測）
   - 局部異常檢測
5. **訓練數據**: 為什麼異常檢測通常只用正常數據訓練?
6. **誤報問題**: 如果模型產生過多誤報（False Positive），如何改善?
7. **工業應用**: 在實際化工製程中部署異常檢測系統時，有哪些挑戰?
8. **模型更新**: 製程條件改變時，異常檢測模型應該如何更新?

---
# 想對老師說的話